In [ ]:
# Cell 1: Install dependencies
!pip install transformers datasets scikit-learn pandas torch
!pip install huggingface_hub[hf_xet]
!pip install transformers[torch]

In [ ]:
# Cell 2: Import required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
)
import torch
import os
import zipfile
import shutil

In [ ]:
# Cell 3: Load data
ZIP_PATH = os.path.join("engine", "data", "processed_cyberbullying_dataset.zip")

EXTRACT_DIR = os.path.join("engine", "data")
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(EXTRACT_DIR)

DATA_DIR = os.path.join("engine", "data", "processed_cyberbullying_dataset.csv")
assert os.path.exists(DATA_DIR), f"Data file not found: {DATA_DIR}"
df = pd.read_csv(DATA_DIR)
df = df[["text", "binary_label"]]
df["label"] = df["binary_label"].map({"not_cyberbullying": 0, "cyberbullying": 1})
df.head()

In [ ]:
# Cell 4: Tokenization
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

def tokenize(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

In [ ]:
# Cell 5: Dataset preparation
train_texts, val_texts = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
train_ds = Dataset.from_pandas(train_texts[["text", "label"]])
val_ds = Dataset.from_pandas(val_texts[["text", "label"]])

train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)

train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Cell 6: Load pre-trained model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

In [ ]:
# Cell 7: Handle class imbalance with weights
class_counts = df["label"].value_counts().sort_index().values
total = class_counts.sum()
weights = torch.tensor([total / (2 * c) for c in class_counts], dtype=torch.float)

In [ ]:
# Cell 8: Define custom Trainer with weighted loss
from transformers import Trainer

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss(weight=weights.to(logits.device))
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
# Cell 9: Evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

In [ ]:
# Cell 10: Define training arguments
MODEL_DIR = os.path.join("model")

training_args = TrainingArguments(
    output_dir=MODEL_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="logs",
    logging_steps=100
)

In [ ]:
# Cell 11: Initialize and train the model
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

In [ ]:
# Cell 12: Rename labels 
if os.path.exists(MODEL_DIR):
    shutil.rmtree(MODEL_DIR)
    
model.config.id2label = {0: "not_cyberbullying", 1: "cyberbullying"}
model.config.label2id = {"not_cyberbullying": 0, "cyberbullying": 1}

In [ ]:
# Cell 13: Save model and tokenizer
model.save_pretrained(MODEL_DIR)
tokenizer.save_pretrained(MODEL_DIR)

In [ ]:
# Cell 14: Verify Classifier Weights
print(f"Classifier Weights: {model.classifier.out_proj.weight.mean().item()}")